#                                     Text Generation Project with Poem dataset by Laurence Moroney



In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras.utils as ku
from tensorflow.keras.layers import Dense, Flatten, Bidirectional, LSTM, Dropout, Embedding
from tensorflow.keras.models import Sequential


In [2]:
data = open('sonnets.txt').read()  #Reading data file
corpus = data.lower().split('\n')  #Converting the alphabets to small and splitting data from where '\n' appear

tokenizer = Tokenizer()  
tokenizer.fit_on_texts(corpus)  #feeding vocab to the model
total_words = len(tokenizer.word_index) + 1
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]  #converting text to the sequences
    for i in range(1 , len(token_list)):
        n_gram_sequences = token_list[:i+1]  #converting token list to n_gram_sequences
        input_sequences.append(n_gram_sequences)  #feeding N-gram_sequences to a separate list named as 'input_sequences'

maximum_len = max([len(x) for x in input_sequences])  #getting length of the biggest sentence in the 'input_sequences'
input_sequences = pad_sequences(input_sequences , maxlen = maximum_len , padding = 'pre')  #pre_padding the 'input_sequences'

predictors , label = input_sequences[:,:-1],input_sequences[:,-1]  #getting labels and predictors for the 'input_sequences'
label = ku.to_categorical(label , num_classes = total_words)  #labelling entire vocabullary uniquely

In [3]:
model = Sequential()  

model.add(Embedding(total_words , 100 , input_length = maximum_len-1))  #Embedding with '100' dims
model.add(Bidirectional(LSTM(150 , return_sequences  = True)))  #'LSTM' with '150' units.
model.add(Dropout(0.2))  #dropping '20%' of the neurons.
model.add(LSTM(100))  #'LSTM' of 100 units.
model.add(Flatten())
model.add(Dense(total_words/2.00 ,activation =  'relu', kernel_regularizer = tf.keras.regularizers.l2(0.01)))
model.add(Dense(total_words , activation = 'softmax'))

model.compile(optimizer = 'adam', loss = 'categorical_crossentropy' , metrics = ['accuracy'])

history = model.fit(predictors , label , epochs = 100 , verbose = 1)

Train on 12038 samples
Epoch 1/100
12038/12038 [==============================] - 61s 5ms/sample - loss: 6.8414 - accuracy: 0.0639
Epoch 2/100
12038/12038 [==============================] - 54s 5ms/sample - loss: 6.3066 - accuracy: 0.0663
Epoch 3/100
12038/12038 [==============================] - 52s 4ms/sample - loss: 6.1310 - accuracy: 0.0681
Epoch 4/100
12038/12038 [==============================] - 51s 4ms/sample - loss: 6.0138 - accuracy: 0.0774
Epoch 5/100
12038/12038 [==============================] - 51s 4ms/sample - loss: 5.9022 - accuracy: 0.0811
Epoch 6/100
12038/12038 [==============================] - 53s 4ms/sample - loss: 5.7751 - accuracy: 0.0930
Epoch 7/100
12038/12038 [==============================] - 51s 4ms/sample - loss: 5.6562 - accuracy: 0.1012
Epoch 8/100
12038/12038 [==============================] - 47s 4ms/sample - loss: 5.5580 - accuracy: 0.1080
Epoch 9/100
12038/12038 [==============================] - 51s 4ms/sample - loss: 5.4836 - accuracy: 0.1102
Epoch

In [8]:
seed_text = "Help me Obi Wan Kenobi, you're my only hope"  #using this sentence to generate a poem
next_words = 100  #generating '100' words of poem
  
for _ in range(next_words):
    token_list = tokenizer.texts_to_sequences([seed_text])[0]
    token_list = pad_sequences([token_list], maxlen=maximum_len-1, padding='pre')
    predicted = model.predict_classes(token_list, verbose=0)
    output_word = ""
    for word, index in tokenizer.word_index.items():  #looping through words and indices of seed text.
        if index == predicted:  #condition to get predicted index equal to current index, to use it for next predictions.
            output_word = word  #condition to get output word equal to current word, to use it for next predictions.
            break
    seed_text += " " + output_word  #joining seed text with next predicted text.
print(seed_text)

Help me Obi Wan Kenobi, you're my only hope forever as he gave to me a whirligig that no tenant thou with captain pepper as his fathers barrow till she well tell me home to me to care you love my souls william beguile agree on your breast in the grass he said no tight thou him down and rest to me very as hit left me contented now in my saber forever it doesnt we sat course it had a stone thee clung of the sky above i think i will cease to sip trod the pale sands the cliffs of doneen no black song on the bridge
